In [1]:
import tensorflow.keras as keras
from tensorflow.keras import preprocessing
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import inception_resnet_v2
import numpy as np
import os
import glob
from PIL import Image
import sys

## Load data and preprocess images

In [2]:
# converting all jpg pictures to png format
base_path = './data/'
classes = ['oak', 'maple', 'beech']
for leaf in classes:
    path = base_path + leaf
    for file_name in os.listdir(path):        
        if file_name.endswith('.jpg'):
            im = Image.open(f'{path}/{file_name}')
            last_char_index = file_name.rfind(".jpg")
            new_name = file_name[:last_char_index] + ".png"
            im.save(f'{path}/{new_name}')
            os.remove(os.path.join(path, file_name))

## Pipeline like data upload

In [3]:
# define an image data generator
data_gen_train = preprocessing.image.ImageDataGenerator(
    preprocessing_function=inception_resnet_v2.preprocess_input)

In [4]:
data_gen_test = preprocessing.image.ImageDataGenerator(
    preprocessing_function=inception_resnet_v2.preprocess_input)

In [5]:
# a generator that returns batches of X and y arrays
train_data_gen = data_gen_train.flow_from_directory(
        directory=base_path,
        class_mode="categorical",
        classes=classes,
        batch_size=1200,
        target_size=(224, 224)
)

Found 1159 images belonging to 3 classes.


In [6]:
test_data_gen = data_gen_test.flow_from_directory(
        directory=base_path,
        class_mode="categorical",
        classes=classes,
        batch_size=197,
        target_size=(224, 224)
)

Found 1159 images belonging to 3 classes.


In [7]:
X_train, y_train = next(train_data_gen)
X_train.shape, y_train.shape

((1159, 224, 224, 3), (1159, 3))

In [8]:
X_test, y_test = next(test_data_gen)
X_test.shape, y_test.shape

((197, 224, 224, 3), (197, 3))

## Model pipeline building on inception_resnet_v2

In [9]:
base_model = keras.applications.InceptionResNetV2(
    include_top=False,
    weights="imagenet",
    input_shape=(224, 224, 3),
    pooling='avg',   
)

base_model.trainable = False

In [10]:
model = keras.Sequential()
model.add(base_model)
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(len(classes), activation='softmax'))

In [11]:
%%time
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, 
          epochs=50, 
          verbose=1,
          batch_size=300, 
          callbacks=[callback],
          validation_split=0.2)

Train on 927 samples, validate on 232 samples
Epoch 1/50
927/927 [==============================] - 174s 188ms/sample - loss: 1.2762 - categorical_accuracy: 0.4509 - val_loss: 1.4562 - val_categorical_accuracy: 0.3534
Epoch 2/50
927/927 [==============================] - 152s 164ms/sample - loss: 0.5367 - categorical_accuracy: 0.7864 - val_loss: 0.8909 - val_categorical_accuracy: 0.5302
Epoch 3/50
927/927 [==============================] - 145s 157ms/sample - loss: 0.3587 - categorical_accuracy: 0.8619 - val_loss: 0.7699 - val_categorical_accuracy: 0.6078
Epoch 4/50
927/927 [==============================] - 210s 226ms/sample - loss: 0.3016 - categorical_accuracy: 0.8997 - val_loss: 0.7714 - val_categorical_accuracy: 0.6250
Epoch 5/50
927/927 [==============================] - 154s 166ms/sample - loss: 0.2353 - categorical_accuracy: 0.9213 - val_loss: 0.8906 - val_categorical_accuracy: 0.6078
Epoch 6/50
927/927 [==============================] - 153s 165ms/sample - loss: 0.1873 - categ

In [ ]:
%%time
base_model.trainable = True

model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])
callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

model.fit(X_train, y_train, 
          epochs=10, 
          verbose=1,
          batch_size=300, 
          callbacks=[callback],
          validation_split=0.2)

Train on 927 samples, validate on 232 samples
Epoch 1/10
927/927 [==============================] - 838s 904ms/sample - loss: 0.1552 - categorical_accuracy: 0.9622 - val_loss: 0.9621 - val_categorical_accuracy: 0.6034
Epoch 2/10
927/927 [==============================] - 844s 911ms/sample - loss: 0.1044 - categorical_accuracy: 0.9806 - val_loss: 0.8638 - val_categorical_accuracy: 0.6293
Epoch 3/10
927/927 [==============================] - 791s 853ms/sample - loss: 0.0746 - categorical_accuracy: 0.9914 - val_loss: 0.7930 - val_categorical_accuracy: 0.6552
Epoch 4/10
927/927 [==============================] - 788s 850ms/sample - loss: 0.0529 - categorical_accuracy: 0.9978 - val_loss: 0.7371 - val_categorical_accuracy: 0.6767
Epoch 5/10
927/927 [==============================] - 823s 888ms/sample - loss: 0.0492 - categorical_accuracy: 0.9989 - val_loss: 0.6804 - val_categorical_accuracy: 0.6853
Epoch 6/10


In [16]:
model.save('maple_oak_beech_basemodel_trained.h5')